In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random

2025-11-23 05:32:52.526291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-23 05:32:52.526360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-23 05:32:52.529086: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-23 05:32:52.545766: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 05:32:54.821776: W tensorflow/compiler/tf2

In [2]:
# Set random seeds
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [3]:
## Load the Cleaned Mobile Price Prediction Dataset
dataframe = pd.read_csv("Mobile-Price-Prediction-cleaned_data.csv")
dataframe.head()

,Ratings,RAM,ROM,Mobile_Size,Primary_Cam,Selfi_Cam,Battery_Power,Price
0,4.3,4.0,128.0,6.00,48,13.0,4000,24999
1,3.4,6.0,64.0,4.50,48,12.0,4000,15999
2,4.3,4.0,4.0,4.50,64,16.0,4000,15000
3,4.4,6.0,64.0,6.40,48,15.0,3800,18999
4,4.5,6.0,128.0,6.18,35,15.0,3800,18999


In [4]:
# Get a Concise Summary of the Dataset
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807 entries, 0 to 806
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Ratings        807 non-null    float64
 1   RAM            807 non-null    float64
 2   ROM            807 non-null    float64
 3   Mobile_Size    807 non-null    float64
 4   Primary_Cam    807 non-null    int64  
 5   Selfi_Cam      807 non-null    float64
 6   Battery_Power  807 non-null    int64  
 7   Price          807 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 50.6 KB


In [5]:
# Generate descriptive statistics of the numeric columns in the dataframe
dataframe.describe().T

,count,mean,std,min,25%,50%,75%,max
Ratings,807.0,4.112639,0.367830,2.8,3.9,4.10,4.4,4.8
RAM,807.0,5.941760,2.056359,0.0,6.0,6.00,6.0,12.0
ROM,807.0,64.390335,53.868626,2.0,32.0,32.00,64.0,256.0
Mobile_Size,807.0,5.620660,3.957409,2.0,4.5,4.77,6.3,44.0
Primary_Cam,807.0,47.821561,11.155916,5.0,48.0,48.00,48.0,64.0
Selfi_Cam,807.0,8.868649,4.547254,0.0,8.0,8.00,12.0,23.0
Battery_Power,807.0,3278.859975,937.628801,1020.0,3000.0,3000.00,3800.0,6000.0
Price,807.0,14269.167286,23092.739983,479.0,984.0,1699.00,18994.5,153000.0


In [6]:
# Separating predictors from targets
X = dataframe.drop("Price", axis=1)
y = dataframe["Price"]

In [7]:
## Get the shape of the X and y
X.shape, y.shape

((807, 7), (807,))

In [8]:
# Split into train, val and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((564, 7), (243, 7), (564,), (243,))

In [9]:
## Import the StandardScaler class and standardise (normalise) features 
# by scaling them to a standard normal distribution.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# For efficiency, convert data to tensors or as a tf_.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(128)
val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(128)

2025-11-23 05:32:57.846915: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [11]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [12]:
for batch in train_dataset:
    print(len(batch[0]))

128
128
128
128
52


In [13]:
type(train_dataset)

tensorflow.python.data.ops.batch_op._BatchDataset

In [14]:
input_shape = X_train.shape[1]
input_shape

7

In [15]:
# Define Model using the Sequential API with Input Layers, 2 hidden layers
# (128 and 64 units, ReLU activation) and an Output layer (1 unit)

seq_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_shape, ), name="input_layer"),
    tf.keras.layers.Dense(128, activation="relu", name="dense_layer_one"),
    tf.keras.layers.Dense(64, activation='relu', name="dense_layer_two"),
    tf.keras.layers.Dense(1, name="output_layer") # regression output (1 unit)
])

In [16]:
seq_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer_one (Dense)     (None, 128)               1024      
                                                                 
 dense_layer_two (Dense)     (None, 64)                8256      
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 9345 (36.50 KB)
Trainable params: 9345 (36.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
## Define Optimizer, Loss and Metrics
optimizer = tf.keras.optimizers.Adam(1e-3)
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer, loss_fn

(<keras.src.optimizers.adam.Adam at 0x74bea6a7ed50>,
 <keras.src.losses.MeanSquaredError at 0x74bea6a7ec10>)

In [18]:
# Configure metrics
train_loss_metric = tf.keras.metrics.Mean(name="train_loss")
val_loss_metric = tf.keras.metrics.Mean(name="val_loss")
train_loss_metric, val_loss_metric

(<keras.src.metrics.base_metric.Mean at 0x74bea655da50>,
 <keras.src.metrics.base_metric.Mean at 0x74bea64d0d50>)

In [19]:
# # Custom Training Step Function (TensorFlow)
# # All comments are written below each line

# @tf.function 
# # Compile the function into a TensorFlow graph for speed

# def train_step(x, y):

#     with tf.GradientTape() as tape:
#     # Record the operations for automatic differentiation

#         y_pred = model(x, training=True)
#         # Forward pass: model predicts outputs

#         loss_value = loss_fn(y, y_pred) 
#         # Compute primary loss (MSE defined earlier)

#         loss_value += sum(seq_model.losses)  
#         # Add regularization losses (L2, dropout penalties, etc.) if the model has them

#     grads = tape.gradient(loss_value, seq_model.trainable_variables)
#     # Compute gradients of the total loss w.r.t the model’s trainable parameters

#     optimizer.apply_gradients(zip(grads, seq_model.trainable_variables))
#     # Update the model weights using the optimizer (SGD, Adam, etc.)

#     train_loss_metric.update_state(loss_value)
#     # Track and update the running training loss metric

In [20]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_pred = seq_model(x, training=True)
        loss_value = loss_fn(y, y_pred)
        print("Model Losses: ", seq_model.losses)
        loss_value += sum(seq_model.losses)
    grads = tape.gradient(loss_value, seq_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, seq_model.trainable_variables))
    train_loss_metric.update_state(loss_value)        

In [21]:
# Validation Step unction

@tf.function
def val_step(x, y):
    y_pred = seq_model(x, training=False)
    v_loss = loss_fn(y, y_pred)
    val_loss_metric.update_state(v_loss)

In [22]:
# Training Loop
EPOCHS = 1000

for epoch in range(EPOCHS):

    # Reset metrics at the start of each epoch
    train_loss_metric.reset_states()
    val_loss_metric.reset_states()

    # Training 
    for x_batch, y_batch in train_dataset:
        train_step(x_batch, y_batch)

    # Validation
    for x_batch, y_batch in val_dataset:
        val_step(x_batch, y_batch)

    if (epoch+1)%100 == 0:
        print(f"Epoch: {epoch+1} Train Loss: {train_loss_metric.result():.4f} Val Loss: {val_loss_metric.result():.4f}")

Model Losses:  []
Model Losses:  []
Model Losses:  []
Epoch: 100 Train Loss: 406139040.0000 Val Loss: 495986336.0000
Epoch: 200 Train Loss: 245142880.0000 Val Loss: 319466048.0000
Epoch: 300 Train Loss: 225213312.0000 Val Loss: 308645632.0000
Epoch: 400 Train Loss: 212284784.0000 Val Loss: 303041536.0000
Epoch: 500 Train Loss: 202540976.0000 Val Loss: 298684640.0000
Epoch: 600 Train Loss: 195185184.0000 Val Loss: 295784000.0000
Epoch: 700 Train Loss: 189858976.0000 Val Loss: 294262272.0000
Epoch: 800 Train Loss: 186005408.0000 Val Loss: 293208736.0000
Epoch: 900 Train Loss: 182645152.0000 Val Loss: 291604352.0000
Epoch: 1000 Train Loss: 179003168.0000 Val Loss: 289376448.0000


In [24]:
## Evaluation after training
y_pred = seq_model.predict(X_test)

# Calculate MAE or MSE manually
mse = tf.keras.metrics.mean_squared_error(
    y_test, y_pred).numpy().mean()

mae = tf.keras.metrics.mean_absolute_error(
    y_test, y_pred).numpy().mean()

print("Test MSE:", mse)
print("Test MAE:", mae)

8/8 [==============================] - 0s 5ms/step
Test MSE: 1016244350.0
Test MAE: 20819.068


In [25]:
from sklearn.metrics import r2_score

score = r2_score(y_test, y_pred)
score

0.5220667749392034

In [ ]:
0.5220667749392034